# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#pip install gmaps

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('../WeatherPy/city_data.csv')
city_df.drop(city_df.columns[[0,1]], axis = 1, inplace = True) #needed to drop an extra column created
city_df

City    Lat     Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
0    pacific grove  36.62 -121.92     70.00        56          40       14.99   
1           laguna  38.42 -121.42     84.00        39           1       12.75   
2             miri   4.41  114.01     75.20        94          40        2.24   
3          veraval  20.90   70.37     83.28        82         100       13.29   
4           dikson  73.51   80.55     39.96        93          27        7.76   
..             ...    ...     ...       ...       ...         ...         ...   
568       khorixas -20.37   14.97     60.87        29           0       10.20   
569  santa rosalia  27.32 -112.28     88.95        55          73        7.87   
570          arman  59.70  150.17     59.20        65          34        2.86   
571        bubaque  11.28  -15.83     80.85        82          99       10.31   
572     lavassaare  58.51   24.39     57.00        70           0        8.99   

    Country        Date  
0        US  1595451206  
1        US  1595451206  
2        MY  1595451491  
3        IN  1595451496  
4        RU  1595451476  
..      ...         ...  
568     NaN  1595451694  
569      MX  1595451694  
570      RU  1595451694  
571      GW  1595451695  
572      EE  1595451695  

[573 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) 

In [4]:
locations = city_df[["Lat", "Lng"]].astype(float)
humidity_rate = city_df["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
filteredcity_df = city_df.loc[(city_df["Humidity"] < 19) & 
                              (city_df["Wind Speed"] < 15) & 
                              (city_df["Max Temp"] > 80) & 
                              (city_df["Cloudiness"] < 30)]
#filteredcity_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#added a new column to old df, makes no sense to make a new one with 1 column...
filteredcity_df["Hotel Name"] = ''

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in filteredcity_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    try:
        filteredcity_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


C:\Users\Rajeshkuma\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Rajeshkuma\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


In [10]:
filteredcity_df
hotel_df = filteredcity_df.loc[(filteredcity_df["Hotel Name"] != '')] #Made sure if no results, then it wouldn't include it
hotel_df


City    Lat     Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
75   saint george  37.10 -113.58     96.80        18          20        5.82   
264       sakakah  29.97   40.21     95.00        10           0        4.70   
287       worland  44.02 -107.96     97.00        13           1        8.05   
388       pahrump  36.21 -115.98    102.00        12           0       14.00   
469     hurricane  37.18 -113.29     96.80        18          20        5.82   
514         dahuk  36.87   42.99     86.65        17           0        7.16   
524         aswan  24.09   32.91     89.20        18           0       13.33   

    Country        Date                                 Hotel Name  
75       US  1595451408                Best Western Plus Abbey Inn  
264      SA  1595451267                            Raoum Inn Hotel  
287      US  1595451619  Comfort Inn Worland Hwy 16 To Yellowstone  
388      US  1595451647                             Pahrump Nugget  
469      US  1595451667          Best Western Plus Zion West Hotel  
514      IQ  1595451681                             Dilshad Palace  
524      EG  1595451683          Sofitel Legend Old Cataract Aswan

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
bank_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)


# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(bank_layer)

fig

Figure(layout=FigureLayout(height='420px'))